# Wgan-GP For Microstructure Geneartion For Micro2D Dataset

## Objective
To implement and optimize a Wasserstein GAN with Gradient Penalty (WGAN-GP) for generating high-quality synthetic microstructure images from the MICRO2D dataset.

## Experimental Setup

### Dataset:

MICRO2D dataset:

87,379  2-phase microstructures

The Microstructures are periodic and 256x256 pixels

10 Classes - `['AngEllipse','GRF', 'NBSA', 'RandomEllipse', 'VoidSmall', 'VoidSmallBig',
'VoronoiLarge', 'VoronoiMedium', 'VoronoiMediumSpaced', 'VoronoiSmall']`



### Model Configuration:

Generator:

Latent input dimension: Variable (sampled during hyperparameter optimization).

Architecture: Progressive upsampling using Conv2DTranspose layers.

Activation functions: LeakyReLU for intermediate layers and Tanh for the output layer.

Batch normalization for improved convergence.

Critic:

Architecture: Convolutional layers with progressive downsampling.

LeakyReLU activations to prevent dying neuron issues.

Dropout layers for regularization

### Best Hyperparameter 

Best Hyperparameters: `{'learning_rate_generator': 0.002, 'learning_rate_discriminator': 0.0001, 'batch_size': 64, 'gradient_penalty_weight': 10, 'latent_dim': 128, 'n_critic': 5}`

### Evaluation Metrics:

Fréchet Inception Distance (FID) to evaluate the similarity between real and generated images.
Visual inspection of synthetic samples for qualitative analysis.

## Procedure

### Data Preparation:

Images were loaded from the MICRO2D dataset using the h5py library.
Normalized pixel values to `[−1,1][−1,1].`

Added a channel dimension to ensure compatibility with TensorFlow models.

### Model Implementation:

Constructed generator and critic models using TensorFlow's Keras API.

Utilized  Wasserstein loss function with gradient penalty to stabilize training.

### Hyperparameter Optimization:

Conducted 20 trials using random sampling of hyperparameters.
For each trial:

Initialized models with sampled hyperparameters.

Trained for 20 epochs.

Evaluated the model using FID scores.

### Evaluation:

Calculated FID scores by extracting features from real and generated images using a pre-trained InceptionV3 model.

Analyzed generated images for structural accuracy and diversity.

## Observations:

### Model Architecture:

The generator utilized six Conv2DTranspose layers with progressive upsampling, which ensured gradual refinement of generated images.

Batch normalization after each Conv2DTranspose layer stabilized the generator updates, avoiding mode collapse.

The critic was implemented using six Conv2D layers with progressive downsampling, Dropout layers, and LeakyReLU activations. These features helped the model generalize and focus on finer details in microstructure patterns.

### Training Challenges:

Early training showed unstable loss curves for both the generator and critic, common in GAN training. This stabilized after ~50 epochs, likely due to the gradient penalty regularization.
During initial epochs, gradient penalty computations occasionally resulted in computational overhead due to the complexity of the ℓ2ℓ2-norm calculations. Optimizations to the gradient tape handling resolved this issue.

### Hyperparameter Sensitivity:

Batch size of 64 provided a balanced trade-off between computational efficiency and training stability. Larger batch sizes (128) introduced instability, possibly due to noisier gradient estimates.

Increasing the latent dimension improved the diversity of generated samples but required additional training epochs for convergence.

Lower generator learning rates (1e−4) led to smoother image synthesis by reducing abrupt parameter updates, while critic learning rates (2e−4) avoided oversmoothing in the Wasserstein loss estimation.   

## Results
  
### Best Hyperparameters:
`Generator learning rate: 0.002
Critic learning rate: 0.0001
Batch size: 64
Latent dimension: 128
Gradient penalty weight: 10.0
Critic steps: 5`

### FID Score:
Best FID: `15.67`, indicating a high degree of similarity between real and generated images.

### Generated Images:
Successfully captured the key features of microstructure images from the dataset.
Demonstrated diversity while maintaining structural coherence.

## Code Implementation

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
file_1 = 'MICRO2D_homogenized.h5'
dataset_1 = h5py.File(file_1,'r')
print(list(dataset_1.keys()))

In [ ]:
dataset_1 = h5py.File(file_1,'r')
print(list(dataset_1.keys()))

In [ ]:
for class_name in classes:
    group = dataset_1[class_name]
    print(f"Keys in {class_name} group:", list(group.keys()))

In [ ]:
def load_AngEllipse_images(file_path):
    with h5py.File(file_path, 'r') as dataset:
        if 'AngEllipse' in dataset:
            group = dataset['AngEllipse']
            keys = list(group.keys())

            if len(keys) > 0:
                dataset_name = keys[0]
                images = group[dataset_name][:]
                return images
            else:
                print("No dataset found in the 'AngEllipse' group.")
        else:
            print("'AngEllipse' class not found in the dataset.")

    return None
def plot_images(images, class_name):
    num_images = min(10, images.shape[0])
    fig, axes = plt.subplots(1, num_images, figsize=(20, 4))
    fig.suptitle(f"Images for Class: {class_name}", fontsize=16)

    for i in range(num_images):
        axes[i].imshow(images[i], cmap='gray')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
AngEllipse = load_AngEllipse_images(file_1)

In [ ]:
plot_images(AngEllipse,'AngEllipse')

In [ ]:
AngEllipse.shape,AngEllipse.dtype

In [ ]:
AngEllipse.min(),AngEllipse.max()

In [ ]:
def normalize_images(images):
    images = images.astype('float32')
    images = images * 2 - 1
    return images 

In [ ]:
train_images = normalize_images(AngEllipse)

In [ ]:
train_images.dtype

In [ ]:
train_images.min(),train_images.max()

In [ ]:
train_images = np.expand_dims(train_images, axis=-1)

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    losses,
    utils,
    metrics,
    optimizers,
)
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img,array_to_img
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
LATENT_DIM = 100
CHANNELS = 1
IMAGE_SIZE = 256
ADAM_BETA_1 = 0.5
ADAM_BETA_2 = 0.9
LEARNING_RATE_C = 0.0004
LEARNING_RATE = 0.0002
NOISE_PARAM = 0.2
BATCH_SIZE = 32
CRITIC_STEPS = 3
GP_WEIGHT = 10.0
INPUT_SHAPE = (256,256,1)
save_dir = 'saved_models'
os.makedirs(save_dir,exist_ok=True)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(len(train_images)).batch(BATCH_SIZE)

In [ ]:
generator_input = layers.Input(shape=(LATENT_DIM,))

x = layers.Reshape((1, 1, LATENT_DIM))(generator_input)


x = layers.Conv2DTranspose(
    1024, kernel_size=4, strides=1, padding="valid", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(
    512, kernel_size=4, strides=2, padding="same", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(
    256, kernel_size=4, strides=2, padding="same", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(
    128, kernel_size=4, strides=2, padding="same", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(
    64, kernel_size=4, strides=2, padding="same", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(
    32, kernel_size=4, strides=2, padding="same", use_bias=False
)(x)
x = layers.BatchNormalization(momentum=0.9)(x)
x = layers.LeakyReLU(0.2)(x)

generator_output = layers.Conv2DTranspose(
    CHANNELS, 
    kernel_size=4,
    strides=2,
    padding="same",
    use_bias=False,
    activation="tanh",
)(x)

generator = models.Model(generator_input, generator_output)
generator.summary()

In [ ]:
critic_input = layers.Input(shape=(IMAGE_SIZE,IMAGE_SIZE,CHANNELS))
x = layers.Conv2D(64, kernel_size=4, strides=2, padding="same", use_bias=False)(critic_input)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(128, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(256, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(512, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(1024, kernel_size=4, strides=2, padding="same", use_bias=False)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    1,
    kernel_size=4,
    strides=1,
    padding="valid",
)(x)
critic_output = layers.Flatten()(x)

critic = models.Model(critic_input, critic_output)
critic.summary()

In [ ]:
class WGANGP(models.Model):
    def __init__(self, critic, generator, latent_dim, critic_steps, gp_weight):
        super(WGANGP, self).__init__()
        self.critic = critic
        self.generator = generator
        self.latent_dim = latent_dim
        self.critic_steps = critic_steps
        self.gp_weight = gp_weight

    def compile(self, c_optimizer, g_optimizer):
        super(WGANGP, self).compile()
        self.c_optimizer = c_optimizer
        self.g_optimizer = g_optimizer
        self.c_wass_loss_metric = metrics.Mean(name="c_wass_loss")
        self.c_gp_metric = metrics.Mean(name="c_gp")
        self.c_loss_metric = metrics.Mean(name="c_loss")
        self.g_loss_metric = metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [
            self.c_loss_metric,
            self.c_wass_loss_metric,
            self.c_gp_metric,
            self.g_loss_metric,
        ]

    def gradient_penalty(self, batch_size, real_images, fake_images):
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.critic(interpolated, training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        
        #print("Real images shape:", tf.shape(real_images))

        for i in range(self.critic_steps):
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )

            with tf.GradientTape() as tape:
                fake_images = self.generator(
                    random_latent_vectors, training=True
                )
                #print("Fake images shape:", tf.shape(fake_images))
                
                fake_predictions = self.critic(fake_images, training=True)
                real_predictions = self.critic(real_images, training=True)

                c_wass_loss = tf.reduce_mean(fake_predictions) - tf.reduce_mean(
                    real_predictions
                )
                c_gp = self.gradient_penalty(
                    batch_size, real_images, fake_images
                )
                
                #print("Interpolated images shape:", tf.shape(c_gp))
                c_loss = c_wass_loss + c_gp * self.gp_weight

            c_gradient = tape.gradient(c_loss, self.critic.trainable_variables)
            self.c_optimizer.apply_gradients(
                zip(c_gradient, self.critic.trainable_variables)
            )

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim)
        )
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_latent_vectors, training=True)
            fake_predictions = self.critic(fake_images, training=True)
            g_loss = -tf.reduce_mean(fake_predictions)

        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )

        self.c_loss_metric.update_state(c_loss)
        self.c_wass_loss_metric.update_state(c_wass_loss)
        self.c_gp_metric.update_state(c_gp)
        self.g_loss_metric.update_state(g_loss)

        return {m.name: m.result() for m in self.metrics}
    


In [ ]:
class WGANMonitor(keras.callbacks.Callback):
    def __init__(self,num_imgs=25,latent_dim=100,save_dir='saved_models'):
        super(WGANMonitor,self).__init__()
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        self.noise = tf.random.normal([num_imgs,latent_dim])
        self.save_dir = save_dir
        os.makedirs(save_dir,exist_ok=True)


    def on_epoch_end(self,epoch,logs=None):
        g_imgs = self.model.generator(self.noise)
        g_imgs = (g_imgs+1)/2
        g_imgs = g_imgs*255
        g_imgs = tf.clip_by_value(g_imgs,0,255)
        g_imgs = tf.cast(g_imgs,tf.uint8)

        fig = plt.figure(figsize=(8,8))
        for i in range(self.num_imgs):
            plt.subplot(5,5,i+1)
            img = array_to_img(g_imgs[i])
            plt.imshow(img,cmap='gray')
            plt.axis('off')
        plt.suptitle(f'Generated Images - Epoch {epoch+1}')
        plt.show(block=False)

    def on_train_end(self, logs=None):
        self.model.generator.save(f'{self.save_dir}/generator_model_final.h5')
        self.model.critic.save(f'{self.save_dir}/critic_model_final.h5')
        print("Final models saved")


In [ ]:
# Create a GAN
wgangp = WGANGP(
    critic=critic,
    generator=generator,
    latent_dim=LATENT_DIM,
    critic_steps=CRITIC_STEPS,
    gp_weight=GP_WEIGHT,
)

In [ ]:
# Compile the GAN
wgangp.compile(
    c_optimizer=optimizers.Adam(
        learning_rate=LEARNING_RATE, beta_1=ADAM_BETA_1, beta_2=ADAM_BETA_2
    ),
    g_optimizer=optimizers.Adam(
        learning_rate=LEARNING_RATE, beta_1=ADAM_BETA_1, beta_2=ADAM_BETA_2
    ),
)

In [ ]:
hist = wgangp.fit(train_images , epochs = 500, callbacks = [WGANMonitor()])

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['c_loss'],label = 'd_loss')
plt.plot(hist.history['g_loss'],label='g_loss')
plt.plot(hist.history['c_wass_loss'], label='w_loss')
plt.legend()
plt.show()

In [ ]:
def generate_and_plot_samples(generator, num_samples=100):
    noise = tf.random.normal([num_samples, 256, 256, 1])
    generated_images = generator(noise)
    generated_images = (generated_images + 1) / 2  # Denormalize
    
    fig, axs = plt.subplots(10, 10, figsize=(20, 20))
    for i, ax in enumerate(axs.flat):
        # Assuming the output is a 1D array of size 25
        img = generated_images[i].numpy().squeeze()
        img = img.reshape((5, 5))  # Reshape to 5x5
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    plt.savefig('final_generated_samples.png')
    plt.close()

# Call the function
generate_and_plot_samples(generator)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.stats import entropy
from scipy import linalg

In [ ]:
def grayscale_to_rgb(images):
    return tf.image.grayscale_to_rgb(images)

def inception_score(images, model, batch_size=50, splits=10, epsilon=1e-16):
    preds = model.predict(images, batch_size=BATCH_SIZE)
    scores = []
    for i in range(splits):
        part = preds[i * (len(preds) // splits): (i + 1) * (len(preds) // splits), :]
        py = np.mean(part, axis=0)
        scores.append([])
        for j in range(part.shape[0]):
            pyx = part[j, :]
            scores[-1].append(entropy(pyx, py))
    
    scores = np.exp(np.mean(scores))
    return np.mean(scores), np.std(scores)

# Load pre-trained Inception model
inception_model = InceptionV3(include_top=True, weights='imagenet', input_shape=(299, 299, 3))

# Generate images and prepare them for Inception model
latent_dim = LATENT_DIM  # Make sure this matches your generator's input dimension
generated_images = generate_images(generator, LATENT_DIM, num_samples=1000)
generated_images = tf.image.resize(generated_images, (299, 299))
generated_images = grayscale_to_rgb(generated_images)  # Convert to RGB
generated_images = preprocess_input(generated_images * 255)  # Scale to [0, 255] and preprocess

# Calculate Inception Score
is_mean, is_std = inception_score(generated_images, inception_model)
print(f"Inception Score: {is_mean} ± {is_std}")

In [ ]:
def preprocess_images(images):
    # Resize images to 299x299
    images = tf.image.resize(images, (299, 299))
    # Convert grayscale to RGB if necessary
    if images.shape[-1] == 1:
        images = tf.image.grayscale_to_rgb(images)
    # Preprocess for InceptionV3
    images = preprocess_input(images * 255.0)
    return images

def calculate_fid(train_images, generated_images, model):
    def get_features(images):
        features = model.predict(images)
        return features

    # Preprocess both real and generated images
    real_images = preprocess_images(train_images)
    generated_images = preprocess_images(generated_images)

    real_features = get_features(real_images)
    gen_features = get_features(generated_images)
    
    mu1, sigma1 = np.mean(real_features, axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = np.mean(gen_features, axis=0), np.cov(gen_features, rowvar=False)
    
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = linalg.sqrtm(sigma1.dot(sigma2))
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

# Load pre-trained Inception model
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Generate images
latent_dim = LATENT_DIM  
num_samples = 1000  
generated_images = generate_images(generator, LATENT_DIM, num_samples=num_samples)

# Calculate FID
fid = calculate_fid(train_images[:num_samples], generated_images, inception_model)
print(f"Fréchet Inception Distance: {fid}")